In [1]:
%load_ext autotime
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import statsmodels.api as sm
import shapely
from tqdm.contrib.concurrent import process_map
from glob import glob
pd.set_option('display.max_columns', None)

/home/nyou045/retrolens/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
shorelines = gpd.read_file("Data for testing/NZCCDv1.shp")
shorelines

,Region,Site,Digitiser,Scale,Notes,Source,CPS,Proxy,Photoscale,Georef_ER,Pixel_Er,Total_UNCY,USDate,SHLength,Date,ID,geometry
0,Auckland,KarekareBethells,MW,2000,tod,RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,1.265403,2004-01-02,0.0,"LINESTRING Z (1728907.5 5916213.248 0, 1728868..."
1,Auckland,KarekareBethells,MW,2000,tod,RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,0.307010,2004-01-02,1.0,"LINESTRING Z (1729067.838 5914779.733 0, 17290..."
2,Auckland,KarekareBethells,MW,2000,tod (landward),RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,0.255243,2004-01-02,2.0,"LINESTRING Z (1729647.881 5911983.994 0, 17296..."
3,Auckland,KarekareBethells,MW,2000,tod,RL,3,1,40000,5.03,1.416425,5.314890,01/02/2004,1.107251,2004-01-02,3.0,"MULTILINESTRING Z ((1728131.079 5917218.888 0,..."
4,Auckland,KarekareBethells,MW,2000,tod,RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,0.770375,2004-01-02,4.0,"MULTILINESTRING Z ((1729643.228 5912906.682 0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19661,West Coast,WoodpeckerBay,MD,1000,EdgeVeg,RL,2,1,18800,4.00,0.630693,3.736680,11/18/1948,1.191368,1948-11-18,19661.0,"LINESTRING Z (1467237.671 5348414.193 0, 14672..."
19662,Otago,Oamaru,TK,1500,Top of cliff,Max,3,2,0,2.02,0.412236,2.278429,04/04/2018,0.650952,2018-04-04,19662.0,"LINESTRING Z (1444386.944 5008247.367 0, 14443..."
19663,Otago,Oamaru,TK,1500,Top of cliff,Max,3,2,0,2.02,0.412236,2.278429,04/04/2018,0.179434,2018-04-04,19663.0,"LINESTRING Z (1444860.629 5008686.925 0, 14448..."
19664,Otago,Oamaru,TK,1500,Top of cliff,Max,3,2,0,2.02,0.412236,2.278429,04/04/2018,0.174825,2018-04-04,19664.0,"LINESTRING Z (1444996.286 5008822.252 0, 14450..."


In [3]:
CPS_error_lookup = {1: 0.43, 2: 0.73, 3: 0.97, 4: 2.07, 5: 8.59}
def calc_UNCY(row):
  # Calculate Total_UNCY
  Ep = row.Pixel_Er
  Ed = CPS_error_lookup[row.CPS]
  Eg = row.Georef_ER
  return np.sqrt(Ep**2 + Eg**2 + Ed**2)
shorelines["new_Total_UNCY"] = shorelines.apply(calc_UNCY, axis=1)
shorelines["UNCY_diff"] = shorelines.Total_UNCY - shorelines.new_Total_UNCY
shorelines[shorelines.UNCY_diff.abs() > 1].sort_values("UNCY_diff")

,Region,Site,Digitiser,Scale,Notes,Source,CPS,Proxy,Photoscale,Georef_ER,Pixel_Er,Total_UNCY,USDate,SHLength,Date,ID,geometry,new_Total_UNCY,UNCY_diff
4567,Gisborne,TeAraroa,AG,2000,"Landfill, edge of vege",RL,5,1,10800,4.0,0.486351,3.769554,05/19/1939,0.492869,1939-05-19,4567.0,"LINESTRING Z (2074482.21 5821331.308 0, 207447...",9.488131,-5.718577
8670,Northland,Mimiwhangata,ER,1500,None,RL,5,1,16000,4.0,0.255811,4.169225,04/30/1942,1.517729,1942-04-30,8670.0,"LINESTRING Z (1729477.877 6076546.628 0, 17294...",9.479111,-5.309885
8671,Northland,Mimiwhangata,ER,1000,None,RL,5,1,16000,4.0,0.255811,4.169225,04/30/1942,1.031080,1942-04-30,8671.0,"LINESTRING Z (1729601.032 6075467.108 0, 17295...",9.479111,-5.309885
8672,Northland,Mimiwhangata,ER,1000,None,RL,5,1,16000,4.0,0.255811,4.169225,04/30/1942,0.206609,1942-04-30,8672.0,"LINESTRING Z (1729907.188 6075213.925 0, 17299...",9.479111,-5.309885
8669,Northland,Mimiwhangata,ER,2000,None,RL,5,1,16000,4.0,0.255811,4.169225,04/30/1942,0.190708,1942-04-30,8669.0,"LINESTRING Z (1729897.752 6075258.124 0, 17298...",9.479111,-5.309885
6001,Marlborough,CliffordBay,MT,1000,None,LDS,3,1,0,0.0,0.200000,2.071358,01/07/2016,2.012041,2016-01-07,6001.0,"LINESTRING Z (1699954.756 5380156.079 0, 16999...",0.990404,1.080954
6002,Marlborough,CliffordBay,MT,1000,None,LDS,3,1,0,0.0,0.200000,2.071358,01/07/2016,0.045874,2016-01-07,6002.0,"LINESTRING Z (1698434.258 5381486.247 0, 16984...",0.990404,1.080954
6003,Marlborough,CliffordBay,MT,1000,None,LDS,3,1,0,0.0,0.200000,2.071358,01/07/2016,1.002103,2016-01-07,6003.0,"LINESTRING Z (1698359.169 5381604.516 0, 16983...",0.990404,1.080954
6008,Marlborough,CliffordBay,MT,1000,None,LDS,3,1,0,0.0,0.200000,2.071358,01/07/2016,0.158202,2016-01-07,6008.0,"LINESTRING Z (1696569.563 5386097.361 0, 16965...",0.990404,1.080954
6009,Marlborough,CliffordBay,MT,1000,None,LDS,3,1,0,0.0,0.200000,2.071358,01/07/2016,0.540913,2016-01-07,6009.0,"LINESTRING Z (1696538.342 5386277.543 0, 16965...",0.990404,1.080954


In [4]:
#transects = pd.concat(gpd.read_file(f).set_index("Unique_ID") for f in glob("Data for testing/Unique_ID_transects/*.shp"))
transects = gpd.read_file("https://uoa-eresearch.github.io/CoastSat/transects_extended.geojson")
transects = transects[transects.site_id.str.startswith("nzd")].to_crs(2193).drop_duplicates("id").set_index("id")
transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,geometry
id,,,,,,,,,,,,,,,,
nzd0001-0000,nzd0001,359.037136,3197.737936,1.000000,0.050,0.0397,0.0679,-0.179097,181.0,141.0,0.003049,19.669927,576.313696,24.006534,325.783655,"LINESTRING (1596662.375 6190263.089, 1596646.6..."
nzd0001-0001,nzd0001,359.037136,3097.737936,0.968728,NaN,NaN,NaN,-0.353513,181.0,155.0,0.014065,17.893212,493.099724,22.205849,330.813132,"LINESTRING (1596744.903 6190264.499, 1596729.1..."
nzd0001-0002,nzd0001,359.037136,2997.737936,0.937456,0.060,0.0478,0.0776,-0.450910,181.0,158.0,0.029104,16.055484,376.363067,19.400079,336.914059,"LINESTRING (1596827.431 6190265.908, 1596811.6..."
nzd0001-0003,nzd0001,359.037136,2897.737936,0.906184,NaN,NaN,NaN,-0.504308,181.0,159.0,0.045285,14.378447,300.668093,17.339784,345.499340,"LINESTRING (1596909.959 6190267.317, 1596894.2..."
nzd0001-0004,nzd0001,359.037136,2797.737936,0.874912,0.055,0.0450,0.0664,-0.560940,181.0,160.0,0.060820,13.485516,278.536778,16.689421,357.868949,"LINESTRING (1596992.486 6190268.724, 1596976.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nzd0562-0014,nzd0562,NaN,NaN,NaN,0.100,NaN,NaN,0.329676,563.0,378.0,0.121339,5.233794,42.937227,6.552650,193.767109,"LINESTRING (1770293.383 5917676.929, 1770648.7..."
nzd0562-0015,nzd0562,NaN,NaN,NaN,0.100,NaN,NaN,0.141327,563.0,379.0,0.020290,5.767879,52.324543,7.233571,196.718923,"LINESTRING (1770254.942 5917751.29, 1770610.27..."
nzd0562-0016,nzd0562,NaN,NaN,NaN,0.100,NaN,NaN,0.122976,563.0,365.0,0.016935,5.432769,46.516572,6.820306,204.997952,"LINESTRING (1770214.898 5917828.821, 1770573.4..."


In [5]:
def process_row(Unique_ID):
    transect = transects.geometry[Unique_ID]
    # This function reproduces the statistics described in https://pubs.usgs.gov/of/2021/1091/ofr20211091.pdf
    try:
        shorelines["intersect_point"] = shorelines.intersection(transect)
    except:
        print(Unique_ID)
        raise
    df = shorelines[~shorelines.intersect_point.is_empty].sort_values("Date")
    if len(df) < 3:
        #print(f"{Unique_ID} has only {len(df)} shorelines")
        return {}
    df.Date = pd.to_datetime(df.Date)
    df["YearsSinceBase"] = (df.Date - df.Date.min()).dt.days / 365.25
    transect_origin = shapely.get_point(transect, 0)
    df["Distance"] = df.intersect_point.distance(transect_origin)
    LR = sm.OLS(df.Distance, sm.add_constant(df.YearsSinceBase)).fit()
    low, high = LR.conf_int(alpha=.1).loc["YearsSinceBase"]
    LCI = (high - low) / 2
    WLR = sm.WLS(df.Distance, sm.add_constant(df.YearsSinceBase), weights=1 / df.Total_UNCY ** 2).fit()
    low, high = WLR.conf_int(alpha=.1).loc["YearsSinceBase"]
    WCI = (high - low) / 2
    duration = (df.Date.max() - df.Date.min()).days / 365.25
    assert duration > 0, f"{Unique_ID} has a duration of {duration}"
    NSM = df.intersect_point.iloc[0].distance(df.intersect_point.iloc[-1])
    return {
        "UniqueID": Unique_ID,
        "Region": df.Region.value_counts().idxmax(),
        "Start_date": str(df.Date.min().date()),
        "End_date": str(df.Date.max().date()),
        "Duration": round(duration),
        "ShrCount": len(df),
        "NSM": round(NSM, 2),
        "SCE": df.intersect_point.apply(lambda p: df.intersect_point.distance(p).max()).max().round(2),
        "EPR": round(NSM / duration, 2),
        "EPRunc": round(np.sqrt(df.Total_UNCY.iloc[0] ** 2 + df.Total_UNCY.iloc[-1] ** 2) / duration, 2),
        "LRR": LR.params.YearsSinceBase.round(2),
        "LRI": LR.params.const.round(2),
        "LCI": round(LCI, 2),
        "LSE": np.sqrt(LR.mse_resid).round(2),
        "LR2": LR.rsquared.round(2),
        "WLR": WLR.params.YearsSinceBase.round(2),
        "WLI": WLR.params.const.round(2),
        "WCI": round(WCI, 2),
        "WSE": np.sqrt(WLR.mse_resid).round(2),
        "WR2": WLR.rsquared.round(2),
        "geometry": transect,
        "Dates": df.Date.dt.strftime("%Y-%m-%d").tolist(),
        "Distances": df.Distance.round(2).tolist(),
    }

results = process_map(process_row, transects.index, chunksize=100, max_workers=64)
rates = gpd.GeoDataFrame(results, crs=2193)
rates

100%|██████████| 32354/32354 [00:39<00:00, 820.87it/s] 


,UniqueID,Region,Start_date,End_date,Duration,ShrCount,NSM,SCE,EPR,EPRunc,LRR,LRI,LCI,LSE,LR2,WLR,WLI,WCI,WSE,WR2,geometry,Dates,Distances
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
1,nzd0001-0001,Northland,1947-05-26,2021-12-24,75.0,5.0,14.18,23.34,0.19,0.17,-0.10,263.79,0.40,10.67,0.11,0.05,261.99,0.41,1.54,0.03,"LINESTRING (1596744.903 6190264.499, 1596729.1...","[1947-05-26, 1971-02-11, 1980-01-05, 2015-11-0...","[257.48, 265.1, 265.51, 266.64, 243.3]"
2,nzd0001-0002,Northland,1947-05-26,2021-12-24,75.0,5.0,16.86,24.55,0.23,0.17,-0.12,270.56,0.38,10.11,0.15,0.02,269.16,0.38,1.43,0.01,"LINESTRING (1596827.431 6190265.908, 1596811.6...","[1947-05-26, 1971-02-11, 1980-01-05, 2015-11-0...","[267.71, 263.72, 275.41, 271.85, 250.86]"
3,nzd0001-0003,Northland,1947-05-26,2021-12-24,75.0,5.0,17.30,17.30,0.23,0.17,-0.09,278.85,0.33,8.86,0.12,0.05,277.57,0.34,1.29,0.04,"LINESTRING (1596909.959 6190267.317, 1596894.2...","[1947-05-26, 1971-02-11, 1980-01-05, 2015-11-0...","[282.82, 267.74, 278.51, 282.09, 265.52]"
4,nzd0001-0004,Northland,1947-05-26,2021-12-24,75.0,5.0,21.46,23.79,0.29,0.17,-0.15,294.35,0.36,9.57,0.25,-0.05,293.59,0.31,1.17,0.05,"LINESTRING (1596992.486 6190268.724, 1596976.7...","[1947-05-26, 1971-02-11, 1980-01-05, 2015-11-0...","[301.8, 278.01, 290.67, 290.88, 280.34]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
32350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
32351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
32352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN


In [6]:
points = pd.concat(gpd.read_file(f) for f in glob("Data for testing/Unique_ID_points/*.shp"))
points

,Unique_ID,geometry
0,2.010001e+11,POINT Z (1602827.257 5444493.261 0)
1,2.010001e+11,POINT Z (1602821.392 5444501.389 0)
2,2.010001e+11,POINT Z (1602815.442 5444509.464 0)
3,2.010001e+11,POINT Z (1602809.485 5444517.535 0)
4,2.010001e+11,POINT Z (1602803.528 5444525.605 0)
...,...,...
571,1.010019e+11,POINT Z (1784449.725 5926935.288 0)
572,1.010019e+11,POINT Z (1784439.931 5926939.809 0)
573,1.010019e+11,POINT Z (1784431.172 5926944.695 0)
574,1.010019e+11,POINT Z (1784423.016 5926950.491 0)


In [7]:
rates.drop(columns="geometry").to_parquet("rates_with_timeseries_CoastSat_intersects.parquet")


In [8]:
rates_with_timeseries = gpd.GeoDataFrame(rates.drop(columns="geometry").merge(points, left_on="UniqueID", right_on="Unique_ID").drop(columns="Unique_ID"))
rates_with_timeseries["lat"] = rates_with_timeseries.geometry.to_crs(4326).y
rates_with_timeseries["lon"] = rates_with_timeseries.geometry.to_crs(4326).x
rates_with_timeseries.drop(columns="geometry").to_parquet("rates_with_timeseries.parquet")
rates_with_timeseries

ValueError: You are trying to merge on object and float64 columns for key 'UniqueID'. If you wish to proceed you should use pd.concat

In [55]:
rates_with_timeseries.to_excel("rates_with_timeseries.xlsx", index=False)

In [51]:
rates_with_timeseries.UniqueID = rates_with_timeseries.UniqueID.astype(str)
rates_with_timeseries.dtypes

UniqueID        object
Region          object
Start_date      object
End_date        object
Duration       float64
ShrCount       float64
NSM            float64
SCE            float64
EPR            float64
EPRunc         float64
LRR            float64
LRI            float64
LCI            float64
LSE            float64
LR2            float64
WLR            float64
WLI            float64
WCI            float64
WSE            float64
WR2            float64
Dates           object
Distances       object
geometry      geometry
lat            float64
lon            float64
dtype: object

In [53]:
rates_with_timeseries.to_file("rates_with_timeseries.shp")

/home/nyou045/retrolens/venv/lib/python3.10/site-packages/pyogrio/raw.py:709: RuntimeWarning: Value '['1940-04-14', '1949-03-29', '1960-08-19', '1969-04-30', '1980-10-24', '1987-11-17', '2002-04-26', '2009-12-24', '2010-04-07', '2010-11-07', '2011-09-23', '2014-11-29', '2015-03-01', '2016-04-05', '2017-10-23', '2019-03-22', '2020-05-18', '2021-03-21', '2024-04-30']' of field Dates has been truncated to 254 characters.  This warning will not be emitted any more for that layer.
  ogr_write(
